In [ ]:
#install.packages(c("tidyverse", "reshape2"))
# install.packages("fmsb")
library(tidyverse) # loads dplyr, ggplot2, stringr etc
library(reshape2) # for melt an cast

#install.packages(c("car","agricolae", "FSA"))
library(car)
library(agricolae)
library(FSA)

In [ ]:
df <- read.csv("4_benzoxazinoids.csv",
                          header = TRUE)

In [ ]:
# Fold change
control <- df %>% filter(Treatment == "Control") %>% select(Soil, 
                                                            Sample_Type, 
                                                            Benzoxazinoid, 
                                                            Concentration_mg_g)

control <- control %>% group_by(Soil, Sample_Type, Benzoxazinoid) %>% summarise(mean_mg = mean(Concentration_mg_g, na.rm = TRUE))
#names(control)[4] <- "mean_mg"

foldchange <- df %>% filter(Treatment == "Insect")
foldchange <- left_join(foldchange, control, relationship = "many-to-many", by = join_by(Soil, Sample_Type, Benzoxazinoid))
foldchange <- foldchange %>% group_by(Soil, Sample_Type, Benzoxazinoid) %>% mutate(fold_change = Concentration_mg_g / mean_mg)



In [ ]:
##STATS##–––––––– 

# CHECKING NORMAL DISTRIBUTION

# FOR REGULAR DATA -- Check assumptions: normality and homogeneity of variances

shapiro = as.numeric(shapiro.test(df$Concentration_mg_g)$p.value)
levene = as.numeric(car::leveneTest(Concentration_mg_g ~ Soil, data=df)$`Pr(>F)`[1])
if (shapiro > 0.05 & levene > 0.05) {
  print("     you should perform ANOVA test")
} else {
  print("    you should perform Kruskal-Wallis test")
}
print(paste0("Shapiro test p-value: ", shapiro))
print(paste0("Levene test p-value: ", levene))

# FOR LOG-FOLD DATA -- Check assumptions: normality and homogeneity of variances

shapiro = as.numeric(shapiro.test(foldchange$fold_change)$p.value)
levene = as.numeric(car::leveneTest(fold_change ~ Soil, data=foldchange)$`Pr(>F)`[1])
if (shapiro > 0.05 & levene > 0.05) {
  print("you should perform ANOVA test")
} else {
  print("you should perform Kruskal-Wallis test")
}
print(paste0("Shapiro test p-value: ", shapiro))
print(paste0("Levene test p-value: ", levene))


In [ ]:
#Preparation for graphs, 1st 6 BXDs

annotations <- data.frame(a = rep(NA, times = 16), # These are the statistical groups
                          b = rep(NA, times = 16),
                          c = rep(NA, times = 16), 
                          d = rep(NA, times = 16), 
                          e = rep(NA, times = 16), 
                          f = rep(NA, times = 16),
                          a.name = rep(NA, times = 16), # These are the soil names
                          b.name = rep(NA, times = 16),
                          c.name = rep(NA, times = 16),
                          d.name = rep(NA, times = 16),
                          e.name = rep(NA, times = 16),
                          f.name = rep(NA, times = 16))

# Sub df's of the sample and treatment groups

df$condition <- paste0(df$Soil, "_", df$Treatment)
leaf <- df %>% filter(Sample_Type == "Leaf")
root <- df %>% filter(Sample_Type == "Root")

bxd <- unique(df$Benzoxazinoid) # Calculate K-W p-values to determine groups and place those groups into empty dataframe 
                                 # called annotations with corresponding soil. 
                                 # (The agricolae function outputs the results in different soil orders, so it's important
                                 # to ensure that the group letter is listed with the corresponding soil).


stats <- c() # A vector to collect statistics to paste on graph

for (i in bxd[1:6]) {
    grp <- c() # empty vector to collect the statistical groups
    name <- c() # empty vector to collect corresponding soil name

    pos.grp <- which(bxd == i) # where to place group and name information
    pos.name <- which(bxd == i) + 6

    filter.leaf <- leaf %>% filter(Benzoxazinoid == i) # filter for the BXD of interest in each sub df
    filter.root <- root %>% filter(Benzoxazinoid == i)

    kw.leaf <- agricolae::kruskal(filter.leaf$Concentration_mg_g, # Kruskal-Wallis for leaf samples of control treatment
                                filter.leaf$condition, alpha = 0.05, p.adj = "fdr")
    kwres.leaf <- rownames_to_column(kw.leaf$groups, var="Name")
    grp <- c(grp, kwres.leaf$groups)
    name <- c(name, kwres.leaf$Name)

    kw.root <- agricolae::kruskal(filter.root$Concentration_mg_g, # Kruskal-Wallis for leaf samples of insect treatment
                                filter.root$condition, alpha = 0.05, p.adj = "fdr")
    kwres.root <- rownames_to_column(kw.root$groups, var="Name")
    grp <- c(grp, kwres.root$groups)
    name <- c(name, kwres.root$Name)

    stats <- c(stats, i, "Leaf", kw.leaf$statistics, "Root", kw.root$statistics) # collect chisq and p.values to put on graphs
    
    annotations[,pos.grp] <- grp
    annotations[,pos.name] <- name

    }

# Position of annotations
positions <- data.frame(a.pos = rep(c(0.112, 0.175), each = 8), 
                       b.pos = rep(c(0.25, 0.1), each = 8),
                       c.pos = rep(c(0.3, 0.5), each = 8), 
                       d.pos = rep(c(0.02, 0.003), each = 8), 
                       e.pos = rep(c(0.6, 0.2), each = 8),
                       f.pos = rep(c(0.075, 0.15), each = 8))

annotations <- cbind(annotations, positions)

# Adding variables for facets
annotations$Sample_Type <- rep(c("Leaf", "Root"), each = 8)

sample.labs <- c("Leaf samples", "Root samples")
names(sample.labs) <- c("Leaf", "Root")

# Graphing 

for (i in bxd[1:6]) { # length of bxd is 8; the last two BXD's require different treatment
    
    og.name <- names(annotations)[which(bxd == i) + 6] 
    names(annotations)[which(bxd == i) + 6] <- "condition" 

    pos <- which(bxd == i)+12
    lab <- which(bxd == i)

    filtered <- df %>% filter(Benzoxazinoid == i) # filter for the BXD of interest

    plot <- ggplot(filtered, aes(x=condition, y=Concentration_mg_g)) + 
            geom_boxplot(outlier.alpha = 0.8, outlier.shape = 16, aes(fill=Soil)) +
            geom_jitter(alpha=0.9, size=1, aes(shape = Treatment)) + 
            scale_shape_manual(values = c(16,1)) +
            facet_grid(Sample_Type~., scale = "free",  labeller = labeller(Sample_Type = sample.labs)) +
            geom_text(data=annotations, mapping = aes(x = condition, y = annotations[,pos], label = annotations[,lab])) +
            geom_vline(xintercept=c(2.5,4.5,6.5), linetype = "dashed", color = "grey") +
            labs(y = paste0("ng/g ", i)) + 
            scale_fill_manual(values = c("#877cba", "#a35da5", "#69b19a", "#8ec372")) +
            theme_classic() + 
            theme(strip.background = element_rect(fill=NA, color = NA), strip.text = element_text(colour = "black")) +
            theme(axis.line = element_line(color='black'),
                plot.background = element_blank(),
                panel.grid.minor = element_blank(),
                panel.grid.major = element_blank()) +
            theme(strip.text = element_text(size = 12)) + 
            theme(axis.text.x = element_text(angle=45, vjust=1, hjust=1))
    
    print(plot)
  #  ggsave(paste0("~/Desktop/", i, ".pdf", sep = ""))
    names(annotations)[which(bxd == i) + 6] <- og.name # restoring original column name
    }


In [ ]:
# Graphs for lotaustralin and linamarin, which don't have root data
annotations2 <- data.frame(g = rep(NA, times = 8),
                           h = rep(NA, times = 8),                          
                           g.name = rep(NA, times = 8),
                           h.name = rep(NA, times = 8))

for (i in bxd[7:8]) {
    grp <- c()
    name <- c()
    pos.grp <- which(bxd==i)-6
    pos.name <- which(bxd==i)-4

    filter.leaf <- leaf %>% filter(Benzoxazinoid == i) # filter for the BXD of interest
    
    
    kw.leaf <- agricolae::kruskal(filter.leaf$Concentration_mg_g, # Kruskal-Wallis for leaf samples of control treatment
                        filter.leaf$condition, alpha = 0.05, p.adj = "fdr")
    kwres.leaf <- rownames_to_column(kw.leaf$groups, var="Name")
    grp <- c(grp, kwres.leaf$groups)
    name <- c(name, kwres.leaf$Name)


    stats <- c(stats, i, kw.leaf$statistics) # collect chisq and p.values to put on graphs
    
    annotations2[,pos.grp] <- grp
    annotations2[,pos.name] <- name

}

#Position of annotations

positions2 <- data.frame(g.pos = rep(c(0.0225, 0.0225), each = 4), 
                         h.pos = rep(c(0.1, 0.1), each = 4))

annotations2 <- cbind(annotations2, positions2)

In [ ]:
for (i in bxd[7:8]) {
    og.name <- names(annotations2)[which(bxd == i) - 4]

    names(annotations2)[which(bxd == i) - 4] <- "condition"

    pos <- which(bxd == i) - 2
    lab <- which(bxd == i) - 6

    filtered <- df %>% filter(Benzoxazinoid == i & Sample_Type == "Leaf") # filter for the BXD of interest
    
    plot <- ggplot(filtered, aes(x=condition, y=Concentration_mg_g)) + 
            geom_boxplot(outlier.alpha = 0.9, outlier.shape = 16, aes(fill=Soil)) +
            geom_jitter(alpha=0.9, size=1, aes(shape = Treatment)) + 
            scale_shape_manual(values = c(16,1)) +
            geom_text(data=annotations2, mapping = aes(x = condition, y = annotations2[,pos], label = annotations2[,lab])) +
            labs(y = paste0("ng/g ", i)) + 
            geom_vline(xintercept=c(2.5,4.5,6.5), linetype = "dashed", color = "grey") +
            scale_fill_manual(values = c("#877cba", "#a35da5", "#69b19a", "#8ec372")) +
            theme_classic() + 
            theme(strip.text = element_text(size = 12)) + 
            theme(axis.text.x = element_text(angle=45, vjust=1, hjust=1))
    print(plot)
 #   ggsave(paste("~/Desktop/", i, ".pdf", sep = ""))
    names(annotations2)[which(bxd == i) - 4] <- og.name
    }

In [ ]:
# Log-fold change of first six BXs

annotations3 <- data.frame(a = rep(NA, times = 8),
                           b = rep(NA, times = 8),
                           c = rep(NA, times = 8),
                           d = rep(NA, times = 8),
                           e = rep(NA, times = 8),
                           f = rep(NA, times = 8),
                           a.name = rep(NA, times = 8),
                           b.name = rep(NA, times = 8),
                           c.name = rep(NA, times = 8),
                           d.name = rep(NA, times = 8),
                           e.name = rep(NA, times = 8),
                           f.name = rep(NA, times = 8))

#Replacing foldchange NA's with 0 
foldchange <- foldchange %>% mutate(fold_change = ifelse(is.na(fold_change), 0, fold_change))

# Sub df's of the sample and treatment groups
leaf <- foldchange %>% filter(Sample_Type == "Leaf")
root <- foldchange %>% filter(Sample_Type == "Root")

# Calculate K-W p-values to determine groups and place those groups into empty dataframe annotations
# with corresponding soil. (The function outputs the results in different soil orders).

stats1 <- c()

for (i in bxd[1:6]) {
    grp <- c()
    name <- c()
    pos.grp <- which(bxd == i)
    pos.name <- which(bxd == i) + 6

    filter.leaf <- leaf %>% filter(Benzoxazinoid == i)
    filter.root <- root %>% filter(Benzoxazinoid == i)
    
    kw.leaf <- agricolae::kruskal(log2(filter.leaf$fold_change), # Kruskal-Wallis for leaf samples of control treatment
                                  filter.leaf$Soil, alpha = 0.05, p.adj = "fdr")
    kwres.l <- rownames_to_column(kw.leaf$groups, var="Name")
    grp <- c(grp, kwres.l$groups)
    name <- c(name, kwres.l$Name)


    kw.root <- agricolae::kruskal(log2(filter.root$fold_change), # Kruskal-Wallis for root samples of control treatment
                        filter.root$Soil, alpha = 0.05, p.adj = "fdr")
    kwres.r <- rownames_to_column(kw.root$groups, var="Name")
    grp <- c(grp, kwres.r$groups)
    name <- c(name, kwres.r$Name)

    stats1 <- c(stats1, i, "Leaf", kw.leaf$statistics, "Root", kw.root$statistics) # collect chisq and p.values to put on graphs
    
    annotations3[,pos.grp] <- grp
    annotations3[,pos.name] <- name

}

annotations3$Sample_Type <- rep(c("Leaf", "Root"), each = 4)

sample.labs <- c("Leaf samples", "Root samples")
names(sample.labs) <- c("Leaf", "Root")

# Graphing 

for (i in bxd[1:6]) { # length of bxd is 8; the last two BXD's require different treatment
    
    og.name <- names(annotations3)[which(bxd == i) + 6] 
    names(annotations3)[which(bxd == i) + 6] <- "Soil" 

    lab <- which(bxd == i)

    filtered <- foldchange %>% filter(Benzoxazinoid == i) # filter for the BXD of interest

    plot <- ggplot(filtered, aes(x=Soil, y=log2(fold_change), fill = Soil)) + 
            geom_boxplot(outlier.alpha = 0.65, outlier.shape = 17) +
            geom_jitter(alpha=0.5, size=1, shape=17, show.legend = FALSE) + 
            facet_grid(.~Sample_Type, scale = "free",  labeller = labeller(Sample_Type = sample.labs)) +
            geom_text(data=annotations3, mapping = aes(x = Soil, y = 3, label = annotations3[,lab])) +
            labs(y = paste0("log2 fold change ", i), x = "Soil") + 
            scale_fill_manual(values = c("#877cba", "#a35da5", "#69b19a", "#8ec372")) +
            scale_y_continuous(limits = c(-5,3)) +
            geom_hline(yintercept=0, linetype = "dotted") +
            theme_linedraw() + 
            theme(strip.background = element_rect(fill=NA, color = NA), strip.text = element_text(colour = "black")) +
            theme(axis.line = element_line(color='black'),
                plot.background = element_blank(),
                panel.grid.minor = element_blank(),
                panel.grid.major = element_blank()) +
            theme(strip.text = element_text(size = 12))
    
    print(plot)
    #ggsave(paste("~/Desktop/", i, "_foldchange",".pdf", sep =""))
    names(annotations3)[which(bxd == i) + 6] <- og.name # restoring original column name
    }


In [ ]:
#Fold change for linamarin and lotaustralin, leaf only

annotations4 <- data.frame(g = rep(NA, times = 4),
                           h = rep(NA, times = 4),                          
                           g.name = rep(NA, times = 4),
                           h.name = rep(NA, times = 4))

for (i in bxd[7:8]) {
    grp <- c()
    name <- c()
    pos.grp <- which(bxd==i)-6
    pos.name <- which(bxd==i)-4

    filter.l <- foldchange %>% filter(Benzoxazinoid==i)
    
    kw <- agricolae::kruskal(log2(filter.l$fold_change), # Kruskal-Wallis for leaf samples of control treatment
                        filter.l$Soil, alpha = 0.05, p.adj = "fdr")
    kwres <- rownames_to_column(kw$groups, var="Name")
    grp <- c(grp, kwres$groups)
    name <- c(name, kwres$Name)

    stats1 <- c(stats1, i, kw$statistics) # collect chisq and p.values to put on graphs
 
    annotations4[,pos.grp] <- grp
    annotations4[,pos.name] <- name

}

for (i in bxd[7:8]) {
    og.name <- names(annotations4)[which(bxd == i) - 4]

    names(annotations4)[which(bxd == i) - 4] <- "Soil"

    lab <- which(bxd == i) - 6

    filtered <- foldchange %>% filter(Benzoxazinoid == i & Sample_Type == "Leaf") # filter for the BXD of interest
    
    plot <- ggplot(filtered, aes(x=Soil, y=log2(fold_change), fill = Soil)) + 
            geom_boxplot(outlier.alpha = 0.65, outlier.shape = 17) +
            geom_jitter(alpha=0.5, size=1, shape=17, show.legend = FALSE) + 
            geom_text(data=annotations4, mapping = aes(x = Soil, y = 4, label = annotations4[,lab])) +
            labs(y = paste0("log2 fold change ", i), x = "Soil") + 
            scale_fill_manual(values = c("#877cba", "#a35da5", "#69b19a", "#8ec372")) +
            geom_hline(yintercept=0, linetype = "dotted") +
            theme_linedraw()
            
     

    print(plot)
    #ggsave(paste("~/Desktop/", i, "_foldchange", ".pdf", sep = ""))
    names(annotations4)[which(bxd == i) - 4] <- og.name
    }

In [ ]:
#Spider plots
library(fmsb)

spider <- as.data.frame(df %>% group_by(Benzoxazinoid, Sample_Type, Treatment, Soil)
                        %>% summarize(mean = mean(Concentration_mg_g, na.rm = TRUE)))
treatment <- unique(spider$Treatment)
soil <- unique(spider$Soil)
color <- c("#877cba", "#a35da5", "#69b19a", "#8ec372")

# Leaf spider plots
for (i in soil) {
    col <- color[which(soil == i)]
    spider_df <- spider %>% filter(Sample_Type == "Leaf" & Soil == i) %>% select(Benzoxazinoid, Treatment, mean)
    spider_df <- dcast(spider_df, Treatment~Benzoxazinoid) %>% select(-Treatment)
    rownames(spider_df) <- treatment
    spider_df <- rbind(c(0.1, 0.2, 0.2, 0.01, 0.3, 0.04, 0.01, 0.06), rep(0,8), spider_df)

    pdf(file = paste0('~/Desktop/bx_leaf_', i, '.pdf')) 
    radarchart(spider_df, axistype = 1,
          pcol = col,
          plwd = 2.5,
          cglcol = "#000000",
          cglty = 1)
    dev.off()
}

# Root spider plots
for (i in soil) {
    col <- color[which(soil == i)]
    spider_df <- spider %>% filter(Sample_Type == "Root" & Soil == i) %>% select(Benzoxazinoid, Treatment, mean)
    spider_df <- dcast(spider_df, Treatment~Benzoxazinoid) %>% select(-Treatment)
    rownames(spider_df) <- treatment
    spider_df <- rbind(c(0.1, 0.04, 0.4, 4e-4, 0.1, 0.08, 0.01, 0.06), rep(0,8), spider_df) 
    spider_df <- spider_df %>% select(DIBOA_Glc:HMBOA_Glc)

    pdf(file = paste0('~/Desktop/BX_root_', i, '.pdf')) 
    radarchart(spider_df, axistype = 1,
          pcol = col,
          plwd = 2.5,
          cglcol = "#000000",
          cglty = 1)
    dev.off()
}

In [ ]:
# Testing to see whether there is anything interesting in the ratio between DIMBOA and DIMBOA-Glc
# I did DIMBOA:DIMBOA-Glc because DIMBOA-Glc is the storage and I see it as the sort of base


dimboa <- df %>% filter(Benzoxazinoid == "DIMBOA") %>% select(-c(Benzoxazinoid))
dimboa_glc <- df %>% filter(Benzoxazinoid == "DIMBOA_Glc") %>% select(id, Concentration_mg_g) # separate dimboa and dimboa-glc and remove unnecessary columns

together <- left_join(dimboa, dimboa_glc, by = "id")
names(together)[c(7,9)] <- c("dimboa", "dimboa_glc")

together <- together %>% mutate(ratio = dimboa/dimboa_glc) %>% filter(Sample_Type == "Leaf")
head(together)

kw = agricolae::kruskal(log2(together$ratio), 
                    together$condition, alpha = 0.05, p.adj = "fdr")
kwres = rownames_to_column(kw$groups, var="Name")

kw$statistics
plot <- ggplot(together, aes(x=condition, y=log2(ratio))) +
        geom_boxplot(outlier.alpha = 0.65, outlier.shape = 16, aes(fill = Soil)) +
        geom_jitter(alpha=0.5, size = 1, aes(shape = Treatment)) +
        scale_shape_manual(values = c(16,1)) +
        geom_vline(xintercept=c(2.5,4.5,6.5), 
                   linetype = "dashed", color = "grey") +
        geom_hline(yintercept=0, linetype = "dotted") +
        annotate(geom = "text", x=kwres$Name, 
             y = 4, label=kwres$groups,
             color = "#000000") +
        scale_fill_manual(values = c("#877cba", "#a35da5", "#69b19a", "#8ec372")) +
        theme_classic() +
        theme(axis.text.x = element_text(angle=45, vjust=1, hjust=1))

plot

#ggsave("~/Desktop/dimboa_ratio.pdf")